In [32]:
from selenium.webdriver import Chrome
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import re

In [11]:
driver = Chrome()
driver.get('https://twitter.com/i/flow/login')

In [12]:
username = driver.find_element(By.XPATH,'//input[@name = "text"]')
username.send_keys('nad.yul2002@gmail.com')

In [13]:
username.send_keys(Keys.RETURN)

In [14]:
my_password = getpass()

········


In [15]:
password = driver.find_element(By.XPATH, '//input[@name = "password"]')
password.send_keys(my_password)

In [16]:
search_input = driver.find_element(By.XPATH, '//input[@aria-label="Search query"]')
search_input.send_keys('Bloomberg')
search_input.send_keys(Keys.RETURN)

In [19]:
driver.find_element_by_link_text('People').click()
nytimes_input = driver.find_element(By.XPATH, '//span[text() = "Bloomberg"]')
nytimes_input.click()


<ipython-input-19-54116ccab38c>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text('People').click()


In [20]:
cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
card = cards[0]

In [21]:
#Tweet Publisher
card.find_element(By.XPATH, './/span').text

'Bloomberg'

In [22]:
#Tweet Date
card.find_element(By.XPATH, './/time').get_attribute('datetime')

'2022-04-30T09:58:04.000Z'

In [23]:
#Tweet Content 
comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
comment

"Elon Musk's move from California to Texas could potentially save him as much as $1.1 billion on state taxes after selling more than $8.5 billion of Tesla stock to help him buy Twitter\nbloomberg.com\nMusk’s Move to Texas May Yield Big Tax Savings on Tesla Sale\nElon Musk’s relocation from California to Texas has already helped save the world’s richest person a fortune in taxes. Now, after selling more than $8.5 billion of stock to help him buy Twitter Inc.,...\n4\n13\n49"

In [24]:
#Tweet Reply Count
card.find_element(By.XPATH, './/div[@data-testid="reply"]').text

'4'

In [25]:
#Tweet Retweet
card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text

'13'

In [26]:
#Tweet Likes
card.find_element(By.XPATH, './/div[@data-testid="like"]').text

'49'

In [33]:
def get_tweet_data(card):
    """Extract data from tweet"""
    username = card.find_element(By.XPATH, './/span').text
    handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    try:
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
    responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
    text = re.sub(r'\n[0-9]+', '', comment)
    reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text
    retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text
    likes_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, likes_cnt)
    return tweet

In [34]:
get_tweet_data(card)

('Bloomberg',
 '@business',
 '2022-04-30T09:29:06.000Z',
 'As sustainable as mined gems are destructive, Anabela Chan’s jewelry is just as real—and worn by the likes of Rihanna and Taylor Swift\nbloomberg.com\nLab-Grown Gems Have Arrived. Diamond Giants Should Worry\nAs sustainable as mined stones are destructive, Anabela Chan’s jewelry is just as real—and worn by the likes of Rihanna and Taylor Swift.',
 '13',
 '9',
 '20')

In [35]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

tweet_data

[('Bloomberg',
  '@business',
  '2022-04-30T09:58:04.000Z',
  "Elon Musk's move from California to Texas could potentially save him as much as $1.1 billion on state taxes after selling more than $8.5 billion of Tesla stock to help him buy Twitter\nbloomberg.com\nMusk’s Move to Texas May Yield Big Tax Savings on Tesla Sale\nElon Musk’s relocation from California to Texas has already helped save the world’s richest person a fortune in taxes. Now, after selling more than $8.5 billion of stock to help him buy Twitter Inc.,...",
  '6',
  '16',
  '63'),
 ('Bloomberg',
  '@business',
  '2022-04-30T09:44:03.000Z',
  'Germany is backtracking from its G-7 agenda to push for a speedier global exit from coal\nbloomberg.com\nWar Forces Germany to Put G-7 Coal Phaseout on Hold\nGerman Chancellor Olaf Scholz’s government is backtracking from its Group of Seven agenda to push globally for a speedier exit from coal.',
  '8',
  '25',
  '54'),
 ('Bloomberg',
  '@business',
  '2022-04-30T09:29:06.000Z',
 

In [36]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

In [37]:
#get all tweets on the page

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:            
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            if scroll_attempt >=3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break

In [38]:
data

[('Bloomberg',
  '@business',
  '2022-04-30T04:11:40.000Z',
  'U.S. mortgage rates are hovering at their highest level since 2010\nbloomberg.com\nU.S. Mortgage Rates Hover at the Highest Level Since 2010\nU.S. mortgage rates were little changed, holding at the highest level since April 2010.',
  '4',
  '16',
  '40'),
 ('Bloomberg',
  '@business',
  '2022-04-30T03:48:03.000Z',
  'An outpouring of resentment on Chinese social media is overwhelming censors https://trib.al/raSRIuP via \n@BW\nbloomberg.com\nOutpouring of Resentment on Chinese Social Media Is Overwhelming Censors\nNetizens are hijacking official hashtags to sound off on Covid Zero and other government policies.',
  '10',
  '31',
  '45'),
 ('Bloomberg',
  '@business',
  '2022-04-30T03:16:50.000Z',
  'Russia is seeking to move away from the U.S. dollar and rely less on imports in response to sanctions over the war with Ukraine, Chinese state media reported\nbloomberg.com\nRussia to Move Away from U.S. Dollar Amid Sanctions, Xi

In [42]:
with open('C:\\Users\\Jul\\PycharmProjects\\operationResearch\\CSV\\Bloomberg_tweets.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)